In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy.sql import select

# Setup
engine = create_engine('sqlite:///./test.db')
metadata = MetaData()

# Define Table
items = Table(
    'items', metadata,
    Column('id', Integer, primary_key=True, index=True),
    Column('name', String, index=True)
)

# Create Table
metadata.create_all(engine)

# Interacting with Table
with engine.connect() as connection:
    # Insert a new item
    connection.execute(items.insert().values(name='Item1'))
    
    # Query items
    result = connection.execute(select([items]))
    for row in result:
        print(row)


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Setup
engine = create_engine('sqlite:///./test.db')
SessionLocal = sessionmaker(bind=engine)
Base = declarative_base()

# Define ORM Model
class Item(Base):
    __tablename__ = 'items'
    
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)

# Create Tables
Base.metadata.create_all(engine)

# Interacting with ORM Model
def create_item(name: str):
    db = SessionLocal()
    try:
        db_item = Item(name=name)
        db.add(db_item)
        db.commit()
        db.refresh(db_item)
        return db_item
    finally:
        db.close()

def get_items():
    db = SessionLocal()
    try:
        return db.query(Item).all()
    finally:
        db.close()
